In [3]:
using Plots
import Plots: px
theme(:default)
ENV["LINES"] = 20

UndefVarError: UndefVarError: gpu not defined

In [5]:
include("parser.jl")
include("models.jl")
gpu()
download(SIGDataSet)

┌ Info: Precompiling KnetLayers [80bfaf46-ad8a-11e8-19eb-a135e382307b]
└ @ Base loading.jl:1242
ERROR: LoadError: InitError: CUDA error: no CUDA-capable device is detected (code #100, ERROR_NO_DEVICE)
Stacktrace:
 [1] macro expansion at /home/gridsan/eakyurek/.julia/packages/CUDAdrv/ADRHQ/src/base.jl:145 [inlined]
 [2] init(::Int64) at /home/gridsan/eakyurek/.julia/packages/CUDAdrv/ADRHQ/src/init.jl:10 (repeats 2 times)
 [3] __init__() at /home/gridsan/eakyurek/.julia/packages/CUDAdrv/ADRHQ/src/CUDAdrv.jl:83
 [4] _include_from_serialized(::String, ::Array{Any,1}) at ./loading.jl:685
 [5] _require_search_from_serialized(::Base.PkgId, ::String) at ./loading.jl:765
 [6] _tryrequire_from_serialized(::Base.PkgId, ::UInt64, ::String) at ./loading.jl:700
 [7] _require_search_from_serialized(::Base.PkgId, ::String) at ./loading.jl:754
 [8] _require(::Base.PkgId) at ./loading.jl:990
 [9] require(::Base.PkgId) at ./loading.jl:911
 [10] require(::Module, ::Symbol) at ./loading.jl:906
 [11] includ

LoadError: Failed to precompile KnetLayers [80bfaf46-ad8a-11e8-19eb-a135e382307b] to /home/gridsan/eakyurek/.julia/compiled/v1.2/KnetLayers/ZXBRh.ji.

In [ ]:
a = KnetArray(30,5,6)
b = KnetArray(30,5)
vcat(a,b)

In [6]:
data  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-train-high"));
vocab = Vocabulary(data)
edata = encode(data,vocab);
test  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-test"));
tdata = encode(test,vocab);
dictionary = [parseDataLine(line) for line in  eachline("./data/unimorph/spa/spa") if line != ""];

In [7]:
vocab.chars.toElement;

In [8]:
vocab.specialIndices

(unk = 1, mask = 2, eow = 3, bow = 4)

In [9]:
trainsfs = unique(map(x->join(x.surface),data));
testsfs  =  unique(map(x->join(x.surface),test)); 
dictsfs  = unique([map(x->join(x.surface),dictionary); trainsfs; testsfs]);

In [114]:
morph = EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
train_ae!(morph, data, vocab; optim=Adam(), epoch=50)

(loss = 27.164620870971678,)
(loss = 22.277743563842773,)
(loss = 20.867304635620116,)
(loss = 20.033033688354493,)
(loss = 19.3868299407959,)
(loss = 18.674105474853516,)
(loss = 18.08714924621582,)
(loss = 17.50064096221924,)
(loss = 16.92261676635742,)
(loss = 16.388945954895018,)
(loss = 15.922529460144043,)
(loss = 15.494584680175782,)
(loss = 15.070867434692383,)
(loss = 14.722307774353027,)
(loss = 14.350033538818359,)
(loss = 13.996989518737793,)
(loss = 13.751421380615234,)
(loss = 13.464950401306153,)
(loss = 13.235102838134766,)
(loss = 13.027240686035157,)
(loss = 12.85623283996582,)
(loss = 12.69574707183838,)
(loss = 12.539847482299805,)
(loss = 12.468811534118652,)
(loss = 12.55334242401123,)
(loss = 12.421272288513183,)
(loss = 12.333470881652833,)
(loss = 12.258336614990235,)
(loss = 12.157724942016602,)
(loss = 12.133267546081543,)
(loss = 11.93139398651123,)
(loss = 11.699908198547364,)
(loss = 11.61518007659912,)
(loss = 11.564944371032714,)
(loss = 11.4890143188476

(encoder = LSTM{Param{KnetArray{Float32,3}}, Multiply{Param{KnetArray{Float32,2}}}}(input=38,hidden=512,embed=16), Wμ = Multiply{Param{KnetArray{Float32,2}}}(input=512 output=16), Wσ = Dense{ELU}(Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=16), Bias{Param{KnetArray{Float32,1}}}(length=16))), Weaμ = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), Weaσ = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), Wμa = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), Wσa = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), output = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=38), Bias{Param{KnetArray{Float32,1}}}(length=38)), Wdec = Multiply{Param{KnetArray{Float32,2}}}(input=16 output=512), decoder =

In [115]:
morphv =  EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morphv.encoder, morph.encoder)
copytoparams(morphv.Wμ, morph.Wμ)
copytoparams(morphv.Wσa, morph.Wσa)
copytoparams(morphv.Wμa, morph.Wμa)
copytoparams(morphv.Wσ, morph.Wσ)
copytoparams(morphv.Weaμ, morph.Weaμ)
copytoparams(morphv.Weaσ, morph.Weaσ)
morphv.decoder.embedding = morphv.encoder.embedding
train_vae!(morphv, data, vocab; optim=Adam(), epoch=45, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=3)

(kl_weight = 0.1f0, fbr = 3, loss = 24.136080081176758)
(kl_weight = 0.2f0, fbr = 3, loss = 20.495269009399415)
(kl_weight = 0.3f0, fbr = 3, loss = 19.547788580322266)
(kl_weight = 0.4f0, fbr = 3, loss = 18.818262884521484)
(kl_weight = 0.5f0, fbr = 3, loss = 18.21607395324707)
(kl_weight = 0.6f0, fbr = 3, loss = 17.59560764312744)
(kl_weight = 0.70000005f0, fbr = 3, loss = 17.021373539733887)
(kl_weight = 0.8000001f0, fbr = 3, loss = 16.42401690673828)
(kl_weight = 0.9000001f0, fbr = 3, loss = 15.929938021850585)
(kl_weight = 1.0f0, fbr = 3, loss = 15.413370597839355)
(kl_weight = 1.0f0, fbr = 3, loss = 15.019504417419434)
(kl_weight = 1.0f0, fbr = 3, loss = 14.610223217773438)
(kl_weight = 1.0f0, fbr = 3, loss = 14.223151504516602)
(kl_weight = 1.0f0, fbr = 3, loss = 13.919325914001465)
(kl_weight = 1.0f0, fbr = 3, loss = 13.584100263977051)
(kl_weight = 1.0f0, fbr = 3, loss = 13.372099995422364)
(kl_weight = 1.0f0, fbr = 3, loss = 13.091981503295898)
(kl_weight = 1.0f0, fbr = 3, los

In [116]:
samples = sample(morphv, vocab, edata; N=10000);

In [117]:
unique(samples[findall([s ∈ trainsfs for s in samples])])

3628-element Array{String,1}:
 "transfirieres"   
 "imputan"         
 "embaucaré"       
 "espumaras"       
 "invitaran"       
 "selfiaran"       
 "redescubriríamos"
 "interactúen"     
 ⋮                 
 "empobrezcamos"   
 "apropiare"       
 "indexaría"       
 "recolectaran"    
 "os retraéis"     
 "reubicase"       
 "rebuscarán"      

In [118]:
length(samples[findall([s ∈ trainsfs for s in samples])])

5674

In [119]:
unique(samples[findall([s ∈ testsfs for s in samples])])

7-element Array{String,1}:
 "no enamore"   
 "irritaríais"  
 "interpolamos" 
 "desalineará"  
 "retradujere"  
 "conquistareis"
 "civilizase"   

In [120]:
length(samples[findall([s ∈ testsfs for s in samples])])

7

In [121]:
unique(length(samples[findall([s ∈ dictsfs for s in samples])]))

1-element Array{Int64,1}:
 6765

In [122]:
sampleinter(morphv, vocab, data; N=10)

12-element Array{String,1}:
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"
 "acompasa"

In [123]:
au, sigma, mu = calc_au(morphv,tdata; delta=0.01)

(13, Float32[0.023899008; 0.011107493; … ; 0.034281414; 0.024155574], K32(16,1)[0.06590545⋯])

In [124]:
mi = calc_mi(morphv,tdata)

0.24139830017089636

In [134]:
calc_ppl(morphv,edata)

2.768850193697965

In [ ]:
atts = attentions(morph,edata, vocab);

In [ ]:
function getsingle(sfs,exs,perms, αs,i) 
    join(vocab.chars[sfs[i]]), map(e->join(vocab.chars[e]),exs[i][perms[i]]), map(x->x[:,perms[i],i],αs)
end

In [ ]:
i = rand(1:length(atts))
s1,e1,α1 =  getsingle(atts[i]...,rand(1:16))
ys = 1:512
h1 = heatmap(e1, ys, α1.αu; size=(1200,800), xtickfont = font(10, "Halvetica"), title= s1 * ", mu", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);
h2 = heatmap(e1, ys, α1.ασ ;size=(1200,800), xtickfont = font(10, "Halvetica"), title=s1 * ", sigma", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);

In [ ]:
println(s1);println(e1)

In [ ]:
h1

# To-Dos
1. Add unrelated surface forms to the examplars too see whether the attention mappings are meaningful
2. Comparing with Normal Vae
3. Different Sampling Function on LM ($top_k$, temperature)
4. Attention in Decoder
5. Attention to the hidden state sequence, not just to the final state.
6. Metric for testing generation quality?

In [10]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [11]:
morph2 = VAE(length(vocab.chars); H=768, E=16, Z=16)
train_ae!(morph2, data, vocab; optim=Adam(), epoch=40);

┌ Warning: k=(Ptr{Nothing} @0x0000145ad3b06a00, 1024) v=16 ni=11
└ @ Knet /home/gridsan/eakyurek/.julia/packages/Knet/IIjk8/src/gcnode.jl:114


(loss = 25.48106213684082,)
(loss = 19.689124320983886,)
(loss = 17.678109156799316,)
(loss = 16.370846723937987,)
(loss = 15.536462135314942,)
(loss = 14.368440203857421,)
(loss = 13.249526008605956,)
(loss = 12.069123878479004,)
(loss = 10.896824002075196,)
(loss = 9.808624589538574,)
(loss = 8.827186461639405,)
(loss = 7.936842854309082,)
(loss = 7.197932009887695,)
(loss = 6.580391456604004,)
(loss = 5.9857867866516115,)
(loss = 5.381615408325195,)
(loss = 4.859548914718628,)
(loss = 4.425895553970337,)
(loss = 3.9908497341156006,)
(loss = 3.480626374435425,)
(loss = 3.1255239784240723,)
(loss = 2.850472118186951,)
(loss = 2.5576387145996096,)
(loss = 2.3368043502807616,)
(loss = 2.186519179534912,)
(loss = 2.1458755438804626,)
(loss = 1.8643888281822205,)
(loss = 1.7929048946380615,)
(loss = 1.6799940519332885,)
(loss = 1.5623530780792236,)
(loss = 1.4947972586631775,)
(loss = 1.386211607837677,)
(loss = 1.2810854886054992,)
(loss = 1.2522531479358674,)
(loss = 1.2778243978977204,

In [30]:
morph3 = VAE(length(vocab.chars); H=768, E=16, Z=16)
copytoparams(morph3.encoder, morph2.encoder)
copytoparams(morph3.Wμ, morph2.Wμ)
copytoparams(morph3.Wσ, morph2.Wσ)
morph3.decoder.embedding = morph3.encoder.embedding
train_vae!(morph3, data, vocab; optim=Adam(), epoch=45, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=4)

┌ Info: Building the CUDAnative run-time library for your sm_70 device, this might take a while...
└ @ CUDAnative /home/gridsan/eakyurek/.julia/packages/CUDAnative/Lr0yj/src/compiler/rtlib.jl:173


(kl_weight = 0.1f0, fbr = 4, loss = 15.270148815917969)
(kl_weight = 0.2f0, fbr = 4, loss = 14.885761770629882)
(kl_weight = 0.3f0, fbr = 4, loss = 15.071471437072754)
(kl_weight = 0.4f0, fbr = 4, loss = 15.637279937744141)
(kl_weight = 0.5f0, fbr = 4, loss = 16.101967961120607)
(kl_weight = 0.6f0, fbr = 4, loss = 16.486581394958495)
(kl_weight = 0.70000005f0, fbr = 4, loss = 16.79835317993164)
(kl_weight = 0.8000001f0, fbr = 4, loss = 17.05850625152588)
(kl_weight = 0.9000001f0, fbr = 4, loss = 17.139191821289064)
(kl_weight = 1.0f0, fbr = 4, loss = 17.23550920562744)
(kl_weight = 1.0f0, fbr = 4, loss = 17.210766073608397)
(kl_weight = 1.0f0, fbr = 4, loss = 16.833934939575194)
(kl_weight = 1.0f0, fbr = 4, loss = 16.420325607299805)
(kl_weight = 1.0f0, fbr = 4, loss = 16.05777875518799)
(kl_weight = 1.0f0, fbr = 4, loss = 15.691811177062988)
(kl_weight = 1.0f0, fbr = 4, loss = 15.344958515930175)
(kl_weight = 1.0f0, fbr = 4, loss = 15.036047290039063)
(kl_weight = 1.0f0, fbr = 4, loss

In [31]:
samples = sample(morph3, vocab, edata; N=10000, useprior=true)

10005-element Array{String,1}:
 "mantengamos"    
 "gangrenare"     
 "sifrancaríamos" 
 "deporaren"      
 "achuchare"      
 "gestionasen"    
 "potencializando"
 "chuchee"        
 ⋮                
 "anope"          
 "desertalases"   
 "humillaríamos"  
 "relumbraríamos" 
 "enolgueces"     
 "judicializarás" 
 "desabotanarán"  

In [32]:
samples[findall([s ∈ trainsfs for s in samples])]

4274-element Array{String,1}:
 "mantengamos"      
 "gangrenare"       
 "potencializando"  
 "no entrevengan"   
 "pilló"            
 "no trinen"        
 "implorabais"      
 "revitalizabas"    
 ⋮                  
 "horizontalizarían"
 "viabilizaste"     
 "atendiera"        
 "realimentó"       
 "despedidos"       
 "humillaríamos"    
 "judicializarás"   

In [33]:
samples[findall([s ∈ testsfs for s in samples])]

13-element Array{String,1}:
 "deshidratemos"
 "deshidratemos"
 "alegaran"     
 "vibrase"      
 "balbuciera"   
 "sistematizaba"
 "opinásemos"   
 "sanitizaba"   
 "convergía"    
 "reseñarías"   
 "apeáremos"    
 "balbuciera"   
 "encantarás"   

In [34]:
samples[findall([s ∈ dictsfs for s in samples])]

5935-element Array{String,1}:
 "mantengamos"    
 "gangrenare"     
 "achuchare"      
 "gestionasen"    
 "potencializando"
 "estregarás"     
 "no entrevengan" 
 "pilló"          
 ⋮                
 "embocarías"     
 "realimentó"     
 "arrobaras"      
 "despedidos"     
 "humillaríamos"  
 "relumbraríamos" 
 "judicializarás" 

In [35]:
inters = sampleinter(morph3, vocab, data; N=10)

12-element Array{String,1}:
 "mareará" 
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"
 "marrarán"

In [36]:
findall([s ∈ dictsfs for s in inters])

12-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [37]:
au, sigma, mu = calc_au(morph3, tdata; delta=0.01)

(16, Float32[0.37393492; 0.33955055; … ; 0.40281734; 0.41686338], K32(16,1)[0.008007906⋯])

In [38]:
mi =  calc_mi(morph3,tdata)

4.445974746704103

In [50]:
calc_ppl(morph3,edata; nsample=500, B=16)

2.7505987928038524

In [40]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [41]:
model = LSTM_LM(length(vocab.chars); H=768, E=16)

(decoder = LSTM{Param{KnetArray{Float32,3}}, Multiply{Param{KnetArray{Float32,2}}}}(input=38,hidden=768,embed=16,dropout=0.4), output = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=768 output=768), Bias{Param{KnetArray{Float32,1}}}(length=768)), hiddenSize = 768)

In [42]:
train_rnnlm!(model, data, vocab; epoch=40, optim=Adam(), B=16)

(loss = 2.565020043563843,)
(loss = 2.031818983268738,)
(loss = 1.91280202293396,)
(loss = 1.8352395069122314,)
(loss = 1.7840724544525146,)
(loss = 1.7377097730636597,)
(loss = 1.6949951625823974,)
(loss = 1.6514041450500487,)
(loss = 1.6078742918014526,)
(loss = 1.5658175846099853,)
(loss = 1.5220457862854004,)
(loss = 1.4798032117843627,)
(loss = 1.4404640241622926,)
(loss = 1.404750998878479,)
(loss = 1.371851237678528,)
(loss = 1.3426123491287232,)
(loss = 1.3129698717117309,)
(loss = 1.2844683826446532,)
(loss = 1.2616477149963379,)
(loss = 1.2366774326324463,)
(loss = 1.2134045652389527,)
(loss = 1.1957246425628663,)
(loss = 1.1736694137573243,)
(loss = 1.1563226512908935,)
(loss = 1.1420525296211244,)
(loss = 1.1263144080162049,)
(loss = 1.1144586671829224,)
(loss = 1.1005128566741944,)
(loss = 1.0922200533866881,)
(loss = 1.0810391907691956,)
(loss = 1.0751349632263183,)
(loss = 1.0672179356575011,)
(loss = 1.0587451880455017,)
(loss = 1.0532062746047974,)
(loss = 1.0481416421

In [43]:
 calc_ppllm(model, tdata)

6.610169732382499

In [44]:
samples = sample(model, vocab; N=10000)

10016-element Array{String,1}:
 "banalizabas"   
 "inscribiese"   
 "aturarían"     
 "consejas"      
 "rainase"       
 "matorizamos"   
 "reconvinieses" 
 "abejonea"      
 ⋮               
 "gilipeen"      
 "agarrotase"    
 "chaqueteaba"   
 "enmarañaréis"  
 "gravases"      
 "desvalorizaría"
 "uturasen"      

In [45]:
samples[findall([s ∈ trainsfs for s in samples])]

5289-element Array{String,1}:
 "banalizabas"   
 "inscribiese"   
 "aturarían"     
 "consejas"      
 "reconvinieses" 
 "abejonea"      
 "injuriaría"    
 "reimpusisteis" 
 ⋮               
 "mullíamos"     
 "no abrace"     
 "agarrotase"    
 "chaqueteaba"   
 "enmarañaréis"  
 "gravases"      
 "desvalorizaría"

In [46]:
samples[findall([s ∈ testsfs for s in samples])]

11-element Array{String,1}:
 "empedrare"    
 "mostremos"    
 "convergía"    
 "descreyésemos"
 "mordemos"     
 "sistematizaba"
 "enrole"       
 "cumplimentaba"
 "sistematizaba"
 "deshidratemos"
 "reunificarían"

In [47]:
samples[findall([s ∈ dictsfs for s in samples])]

6684-element Array{String,1}:
 "banalizabas"   
 "inscribiese"   
 "aturarían"     
 "consejas"      
 "reconvinieses" 
 "abejonea"      
 "injuriaría"    
 "reimpusisteis" 
 ⋮               
 "no abrace"     
 "avizorar"      
 "agarrotase"    
 "chaqueteaba"   
 "enmarañaréis"  
 "gravases"      
 "desvalorizaría"

In [48]:
 calc_ppllm(model, edata)

2.695941727858836